In [1]:
import uuid
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# テーマリストと各テーマの賛成割合（パーセンテージ）
topics = {
    "外国人労働者の受け入れ拡大": 60,
    "子育て支援の充実": 70,
    "インフラ投資の強化": 50,
    "イノベーションの促進": 80,
    "防衛力の強化": 30,
    "憲法９条の改正": 40,
    "再生可能エネルギーの導入促進": 90,
    "エネルギー安全保障の確保": 20,
    "日米同盟の廃止": 10,
    "教育格差の是正": 55,
    "地域資源の活用": 75,
    "働き方の多様化": 65,
    "労働法制の整備": 85,
    "在宅医療の推進": 95,
    "介護人材の確保": 25,
    "医療費の持続可能性確保": 45,
    "サイバーセキュリティの強化": 35,
    "電子政府（e-Government）の推進": 15,
}

surname = pd.read_csv("../data/surnames.csv", encoding="utf-8")["苗字"].tolist()
given_names = pd.read_csv("../data/given_names.csv", encoding="utf-8")["名前"].tolist()

japanese_names = [
    np.random.choice(surname) + np.random.choice(given_names) for _ in range(100000)
]

for topic, agree_percentage in topics.items():
    topics[topic] = agree_percentage * 0.8

prefecture_city = pd.read_csv("../data/prefecture_city_lonlat.csv", encoding="utf-8")
# prefecture_city.to_csv("data/prefecture_city_lonlat.csv", index=False, encoding="utf-8")
# print(prefecture_city_list)
# 人数
num_people = 100000

# 回答日時の範囲を設定（例：過去3ヶ月）
end_date = datetime.now()
start_date = end_date - timedelta(days=90)

# 空のデータフレームを作成
user_data = pd.DataFrame()

# # 各人のIDを追加
# data["ID"] = range(1, num_people + 1)

# 各人の回答日時をランダムに生成
user_data["id"] = pd.Series([uuid.uuid4() for _ in range(num_people)])
user_data["name"] = np.random.choice(japanese_names, size=num_people)
user_data["sex"] = np.random.choice(["男性", "女性"], size=num_people)
user_data["is_male"] = user_data["sex"] == "男性"
user_data = user_data.drop("sex", axis=1)
user_data["age"] = np.random.randint(10, 100, size=num_people)
pref_indices = np.random.randint(0, prefecture_city.shape[0], size=num_people)
pref_sample = prefecture_city.sample(num_people).reset_index(drop=True)
print(pref_sample)
user_data["prefecture"] = pref_sample["都道府県名"]
user_data["city"] = pref_sample["市区町村名"]
user_data["created_at"] = [
    start_date
    + timedelta(
        seconds=np.random.randint(0, int((end_date - start_date).total_seconds()))
    )
    for _ in range(num_people)
]

answer_data = pd.DataFrame()

# 各テーマに対して賛成か反対かをランダムに割り当てる
for topic_id, (topic, agree_percentage) in enumerate(topics.items()):
    opinions = pd.DataFrame()
    # 賛成の割合を計算
    agree_prob = agree_percentage / 100
    # 中立の割合
    neutral_or_oppose_prob = 1 - agree_prob
    # 中立か反対かをランダムに割り当てる
    oppose_prob = np.random.uniform(low=0, high=neutral_or_oppose_prob)
    neutral_prob = neutral_or_oppose_prob - oppose_prob

    opinions["id"] = pd.Series([uuid.uuid4() for _ in range(num_people)])
    # ランダムに意見を生成
    opinions["value"] = np.random.choice(
        [1, 0, -1], size=num_people, p=[agree_prob, neutral_prob, oppose_prob]
    )
    opinions["answered_at"] = user_data["created_at"] + pd.to_timedelta(
        np.random.randint(0, 24 * 60 * 60, size=num_people), unit="s"
    )
    opinions["user_id"] = user_data["id"]
    opinions["topic_id"] = topic_id + 1
    answer_data = pd.concat([answer_data, opinions])


sample_comment = pd.read_csv("../data/sample_comment.csv", encoding="utf-8")

comment_data = pd.DataFrame()
topic_list = list(topics.keys())

for i in range(sample_comment.shape[0]):
    user = np.random.randint(0, user_data.shape[0])
    user_series = user_data.iloc[user]
    comment = pd.DataFrame(
        {
            "id": [uuid.uuid4()],
            "commented_at": user_series["created_at"]
            + pd.to_timedelta(np.random.randint(0, 24 * 60 * 60), unit="s"),
            "user_id": [user_series["id"]],
            "topic_id": [topic_list.index(sample_comment.iloc[i, 0]) + 1],
            "content": [sample_comment.iloc[i, 1]],
            "parent_id": [np.nan],
            "favorite_count": [np.random.randint(0, 2000)],
            "bad_count": [np.random.randint(0, 2000)],
            "is_agree": [np.nan],
        }
    )
    comment_data = pd.concat([comment_data, comment])

# 回答日時をISO形式の文字列に変換（オプション）
answer_data["answered_at"] = answer_data["answered_at"].dt.strftime("%Y-%m-%d %H:%M:%S")
user_data["created_at"] = user_data["created_at"].dt.strftime("%Y-%m-%d %H:%M:%S")
comment_data["commented_at"] = comment_data["commented_at"].dt.strftime(
    "%Y-%m-%d %H:%M:%S"
)

user_data.to_csv("../data/users.csv", index=False, encoding="utf-8")
answer_data.to_csv("../data/answers.csv", index=False, encoding="utf-8")
comment_data.to_csv("../data/comments.csv", index=False, encoding="utf-8")

       都道府県コード 都道府県名  市区町村コード    市区町村名      大字町丁目コード   大字町丁目名         緯度  \
0            4   宮城県     4103   仙台市若林区   41030063000     かすみ町  38.241257   
1            1   北海道     1226      砂川市   12260033004  西八条北四丁目  43.495973   
2           28   兵庫県    28209      豊岡市  282090052000        辻  35.514917   
3           27   大阪府    27231    大阪狭山市  272310016004   池之原四丁目  34.506541   
4           23   愛知県    23220      稲沢市  232200285000     竹腰東町  35.257063   
...        ...   ...      ...      ...           ...      ...        ...   
99995       12   千葉県    12221     八千代市  122210018001   下市場一丁目  35.716798   
99996       26   京都府    26109   京都市伏見区  261090484000  向島吹田河原町  34.925340   
99997       30  和歌山県    30428  東牟婁郡串本町  304280021000       田並  33.486810   
99998        1   北海道     1211      網走市   12110041002  北一条西二丁目  44.023109   
99999       39   高知県    39205      土佐市  392050013000      積善寺  33.474678   

               経度  原典資料コード  大字・字・丁目区分コード  
0      140.922104        0             1  
1

In [27]:
import os
import pandas as pd
from src.const import DATA_DIR

answer_city = answer_data.merge(
    user_data[["id", "prefecture", "city"]], left_on="user_id", right_on="id"
)
answer_city = answer_city[answer_city["prefecture"] == "東京都"].drop(
    "prefecture", axis=1
)
answer_city["value"] = answer_city["value"].replace({1: "賛成", 0: "中立", -1: "反対"})
answer_by_city = (
    answer_city.groupby(["city", "topic_id", "value"]).size().unstack().fillna(0)
)
answer_by_city = answer_by_city.reset_index()
answer_by_city = answer_by_city.merge(
    pd.read_csv(os.path.join(DATA_DIR, "topics.csv")),
    left_on="topic_id",
    right_on="id",
    how="left",
)
answer_by_city = answer_by_city.drop(["id", "topic_id"], axis=1)
# answer_by_city = answer_by_city
# answer_by_city.to_csv(os.path.join(DATA_DIR, "answer_by_city.csv"), encoding="utf-8")

answer_by_city.merge(
    pd.read_csv(os.path.join(DATA_DIR, "tokyo_election_area.csv")),
    left_on="city",
    right_on="市区町村",
    how="left",
).drop("市区町村", axis=1).set_index(["city", "topic"]).to_csv(
    os.path.join(DATA_DIR, "answer_by_tokyo_election_area.csv"), encoding="utf-8"
)

In [15]:
import sqlite3
import pandas as pd
import os

# データベースファイルのパス
DATA_DIR = "../data"
DB_PATH = DATA_DIR + "/database.db"

# CSVファイルのディレクトリ

# テーブル作成用SQL
CREATE_TABLE_QUERIES = {
    "users": """
        CREATE TABLE IF NOT EXISTS users (
            id TEXT PRIMARY KEY,
            name TEXT NOT NULL,
            is_male INTEGER NOT NULL,
            age INTEGER NOT NULL,
            prefecture TEXT NOT NULL,
            city TEXT NOT NULL,
            created_at DATETIME DEFAULT (datetime('now','localtime')) NOT NULL
        );
    """,
    "topics": """
        CREATE TABLE IF NOT EXISTS topics (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            topic TEXT NOT NULL
        );
    """,
    "comments": """
        CREATE TABLE IF NOT EXISTS comments (
            id TEXT PRIMARY KEY,
            commented_at DATETIME DEFAULT (datetime('now','localtime')) NOT NULL,
            user_id TEXT NOT NULL,
            topic_id INTEGER NOT NULL,
            content TEXT NOT NULL,
            parent_id TEXT,
            favorite_count INTEGER NOT NULL,
            bad_count INTEGER NOT NULL,
            is_agree INTEGER,
            FOREIGN KEY(user_id) REFERENCES users(id),
            FOREIGN KEY(topic_id) REFERENCES topics(id),
            FOREIGN KEY(parent_id) REFERENCES comments(id)
        );
    """,
    "answers": """
        CREATE TABLE IF NOT EXISTS answers (
            id TEXT PRIMARY KEY,
            value INTEGER NOT NULL,
            answered_at DATETIME DEFAULT (datetime('now','localtime')) NOT NULL,
            user_id TEXT NOT NULL,
            topic_id INTEGER NOT NULL,
            FOREIGN KEY(user_id) REFERENCES users(id),
            FOREIGN KEY(topic_id) REFERENCES topics(id)
        );
    """,
}


def enable_foreign_keys(conn):
    """SQLiteではデフォルトで外部キー制約が無効化されているため、有効化します。"""
    conn.execute("PRAGMA foreign_keys = ON;")


def create_tables(conn):
    """必要なテーブルを作成します。"""
    for table_name, query in CREATE_TABLE_QUERIES.items():
        conn.execute(query)
    conn.commit()


def import_csv_to_sqlite(conn, table_name, csv_path, dtype=None, parse_dates=None):
    """
    CSVファイルを読み込み、指定されたSQLiteテーブルにデータを挿入します。

    Parameters:
    - conn: SQLiteのコネクションオブジェクト
    - table_name: データを挿入するテーブル名
    - csv_path: CSVファイルのパス
    - dtype: 列のデータ型（必要に応じて）
    - parse_dates: 日付として解析する列のリスト（必要に応じて）
    """
    print(f"Importing {csv_path} into {table_name} table...")
    df = pd.read_csv(csv_path, dtype=dtype, parse_dates=parse_dates)

    # 日付を文字列に変換（SQLiteはTEXTとして保存）
    if parse_dates:
        for date_col in parse_dates:
            df[date_col] = df[date_col].apply(
                lambda x: x if pd.isnull(x) else pd.to_datetime(x)
            )

    # データをSQLiteに挿入
    df.to_sql(table_name, conn, if_exists="append", index=False)
    print(f"Imported {len(df)} records into {table_name} table.\n")


def main():
    # データベースが存在しない場合はディレクトリを作成
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

    # SQLiteに接続（データベースが存在しない場合は新規作成）
    conn = sqlite3.connect(DB_PATH)
    enable_foreign_keys(conn)
    create_tables(conn)

    # 各テーブルへのCSVインポート
    # users.csv
    import_csv_to_sqlite(
        conn=conn,
        table_name="users",
        csv_path=os.path.join(DATA_DIR, "users.csv"),
        parse_dates=["created_at"],
    )

    # topics.csv
    import_csv_to_sqlite(
        conn=conn, table_name="topics", csv_path=os.path.join(DATA_DIR, "topics.csv")
    )

    # comments.csv
    import_csv_to_sqlite(
        conn=conn,
        table_name="comments",
        csv_path=os.path.join(DATA_DIR, "comments.csv"),
        parse_dates=["commented_at"],
    )

    # answers.csv
    import_csv_to_sqlite(
        conn=conn,
        table_name="answers",
        csv_path=os.path.join(DATA_DIR, "answers.csv"),
        dtype={"value": "Int64"},
        parse_dates=["answered_at"],
    )

    # コネクションを閉じる
    conn.close()
    print("All data has been imported successfully.")


if __name__ == "__main__":
    main()

!rm ../data/users.csv ../data/answers.csv ../data/comments.csv

Importing ../data/users.csv into users table...


FileNotFoundError: [Errno 2] No such file or directory: '../data/users.csv'